In [2]:
from pathlib import Path
import sys

home_path = str(Path.cwd().parent)
sys.path.append(home_path)

In [3]:
from aper_package.interactive_tool import InteractiveTool

In [4]:
tool = InteractiveTool(spark=spark)

In [5]:
tool.show()